<a href="https://colab.research.google.com/github/lobobranco96/pokemon-data-project/blob/main/pokemon-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


EXTRACT

In [3]:
def fetch_pokemon_from_api():
    """
    Busca dados de Pokémon na PokeAPI, especificamente recuperando as gerações disponíveis
    e suas respectivas espécies de Pokémon.

    Retorna:
        dict: Um dicionário contendo duas listas:
            - "region": Nomes das regiões correspondentes a cada geração.
            - "pokemon_id": IDs dos Pokémon em cada geração.
    """

    def find_geracao_available():
        """
        Faz uma requisição à PokeAPI para obter todas as gerações de Pokémon disponíveis.

        Retorna:
            list: Lista com os nomes das gerações.
        """
        response = requests.get("https://pokeapi.co/api/v2/generation/", timeout=5)
        pokemon = response.json()
        return [pokemon_name['name'] for pokemon_name in pokemon.get('results')]

    def fetch_geracao(geracao):
        """
        Obtém dados de uma geração específica, incluindo a região e IDs dos Pokémon.

        Args:
            geracao (str): Nome da geração a ser buscada.

        Retorna:
            tuple: Região correspondente e lista de IDs dos Pokémon dessa geração.
        """
        response = requests.get(f"https://pokeapi.co/api/v2/generation/{geracao}", timeout=5)
        pokemon = response.json()
        regiao = pokemon.get('main_region')['name']
        ids = [nome['url'].split('/')[-2] for nome in pokemon.get('pokemon_species')]
        return regiao, ids

    # Inicializa um dicionário para armazenar os resultados das gerações
    data = {"region": [], "pokemon_id": []}
    for geracao in find_geracao_available():
        regiao, ids = fetch_geracao(geracao)
        data['region'].append(regiao)
        data['pokemon_id'].append(ids)

    return data

def fetch_poke(region, pokemon_id):
    """
    Busca dados detalhados de um Pokémon específico a partir da PokeAPI
    e retorna um dicionário contendo suas informações.

    Args:
        region (str): Região do Pokémon.
        pokemon_id (int): ID do Pokémon a ser buscado.

    Retorna:
        dict: Dicionário com informações detalhadas do Pokémon.
    """
    # Inicializa um dicionário para armazenar informações do Pokémon
    pokemon = {
        'id': None,
        'name': None,
        'region': region,
        'height': None,
        'weight': None,
        'base_experience': None,
        'types': [],
        'abilities': [],
        'moves': [],
        'location_area_encounters': [],
        'held_items': [],
        'base_happiness': None,
        'color': None,
        'capture_rate': None,
        'evolves_from': None,
        'is_baby': None,
        'is_mythical': None,
        'is_legendary': None
    }

    try:
        # Faz a requisição para obter dados do Pokémon
        response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}", timeout=5)
        response.raise_for_status()  # Lança um erro se a requisição falhar
        pokes = response.json()

        # Armazena informações básicas do Pokémon
        pokemon['id'] = pokes.get('id')
        pokemon['name'] = pokes.get('name')
        pokemon['height'] = pokes.get('height')
        pokemon['weight'] = pokes.get('weight')
        pokemon['base_experience'] = pokes.get('base_experience')
        pokemon['abilities'] = [ability['ability']['name'] for ability in pokes.get('abilities', [])]
        pokemon['moves'] = [move['move']['name'] for move in pokes.get('moves', [])]
        pokemon['held_items'] = [item['item']['name'] for item in pokes.get('held_items', [])] or 'Não encontrado.'
        pokemon['types'] = [type['type']['name'] for type in pokes.get('types', [])]

        # Armazena os stats do Pokémon
        for stat in pokes.get('stats', []):
            pokemon[stat['stat']['name']] = stat['base_stat']

        # Função interna para buscar áreas/respaw onde o Pokémon pode ser encontrado
        def fetch_pokemon_areas(id_pokemon):
            """
            Busca as áreas onde o Pokémon pode ser encontrado.

            Args:
                id_pokemon (int): ID do Pokémon a ser buscado.

            Retorna:
                list: Lista de áreas de encontro do Pokémon.
            """
            response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{id_pokemon}/encounters")
            poke_area = response.json()
            return [entry['location_area']['name'] for entry in poke_area]

        # Atualiza o dicionário com as áreas de encontro
        pokemon['location_area_encounters'] = fetch_pokemon_areas(pokemon['id'])

        # Função interna para buscar detalhes adicionais do Pokémon
        def pokemon_details(pokemon_id):
            """
            Busca detalhes adicionais de um Pokémon a partir de seu nome.

            Args:
                pokemon_name (str): Nome do Pokémon.

            Retorna:
                dict: Dicionário com detalhes adicionais do Pokémon.
            """
            response = requests.get(f"https://pokeapi.co/api/v2/pokemon-species/{pokemon_id}")
            pokemon_species = response.json()
            evolves_from = pokemon_species.get('evolves_from_species')
            evolves_from_name = evolves_from['name'] if evolves_from else 'Não encontrado.'

            return {
                'base_happiness': pokemon_species.get('base_happiness'),
                'color': pokemon_species.get('color', {}).get('name'),
                'capture_rate': pokemon_species.get('capture_rate'),
                'evolves_from': evolves_from_name,
                'is_baby': pokemon_species.get('is_baby'),
                'is_mythical': pokemon_species.get('is_mythical'),
                'is_legendary': pokemon_species.get('is_legendary')
            }

        # Obtém os detalhes adicionais do Pokémon
        detail = pokemon_details(pokemon_id)
        pokemon['base_happiness'] = detail['base_happiness']
        pokemon['color'] = detail['color']
        pokemon['capture_rate'] = detail['capture_rate']
        pokemon['evolves_from'] = detail['evolves_from']
        pokemon['is_baby'] = detail['is_baby']
        pokemon['is_mythical'] = detail['is_mythical']
        pokemon['is_legendary'] = detail['is_legendary']

    except requests.exceptions.RequestException as e:
        # Trata erros de requisição
        print(f"Error fetching Pokémon data (ID: {pokemon_id}, Region: {region}): {e}")
        response.raise_for_status()
    return pokemon

# Loop pelas regiões disponíveis no dicionário completo
complete_dicionario = fetch_pokemon_from_api()

for region_index in range(len(complete_dicionario['region'])):
    all_pokemon_data = []  # Lista temporária para armazenar os dados dos Pokémon
    region = complete_dicionario['region'][region_index]

    # Loop pelos IDs dos Pokémon na região atual
    for pokemon_id in complete_dicionario['pokemon_id'][region_index]:
      pokemon_data = fetch_poke(region, pokemon_id)
      all_pokemon_data.append(pokemon_data)  # Adiciona os dados à lista

    # Cria o DataFrame apenas uma vez com todos os Pokémon da região
    dataframe = pd.DataFrame(all_pokemon_data)

    diretorio = "/content/drive/MyDrive/pokemon_region/"
    nome_arquivo = f"{region}.json"
    caminho_arquivo = os.path.join(diretorio, nome_arquivo)
    dataframe.to_json(caminho_arquivo, orient='records', lines=True)
    print(f'Arquivo: {nome_arquivo} salvo com sucesso.')


Arquivo: kanto.json salvo com sucesso.
Arquivo: johto.json salvo com sucesso.
Arquivo: hoenn.json salvo com sucesso.
Arquivo: sinnoh.json salvo com sucesso.
Arquivo: unova.json salvo com sucesso.
Arquivo: kalos.json salvo com sucesso.
Arquivo: alola.json salvo com sucesso.
Arquivo: galar.json salvo com sucesso.
Arquivo: paldea.json salvo com sucesso.


In [32]:
def merge_pokemon_data():
    diretorio = "/content/drive/MyDrive/pokemon_region"
    arquivos = os.listdir(diretorio)
    regioes_disponiveis = arquivos[1:] # o index 0 é uma string desnecessaria para o funcionamento do código
    dataframe = pd.DataFrame()

    for region_name in regioes_disponiveis:
        try:
            # Lê o arquivo JSON
            df = pd.read_json(f'{diretorio}/{region_name}', lines=True)
            # Concatena os dataframe lidos a partir do arquivo json
            dataframe = pd.concat([dataframe, df], ignore_index=True)
        except ValueError as e:
            print(f"Erro ao ler o arquivo {region_name}: {e}")

    return dataframe

# Chama a função e armazena o resultado
pokemon_data = merge_pokemon_data()


In [33]:
pokemon_data

,id,name,region,height,weight,base_experience,types,abilities,moves,location_area_encounters,...,evolves_from,is_baby,is_mythical,is_legendary,hp,attack,defense,special-attack,special-defense,speed
0,1,bulbasaur,kanto,7,69,64,"[grass, poison]","[overgrow, chlorophyll]","[razor-wind, swords-dance, cut, bind, vine-whi...","[cerulean-city-area, pallet-town-area, lumiose...",...,Não encontrado.,False,False,False,45,49,49,65,65,45
1,4,charmander,kanto,6,85,62,[fire],"[blaze, solar-power]","[mega-punch, fire-punch, thunder-punch, scratc...","[pallet-town-area, kanto-route-24-area, lumios...",...,Não encontrado.,False,False,False,39,52,43,60,50,65
2,7,squirtle,kanto,5,90,63,[water],"[torrent, rain-dish]","[mega-punch, ice-punch, mega-kick, headbutt, t...","[vermilion-city-area, pallet-town-area, lumios...",...,Não encontrado.,False,False,False,44,48,65,50,64,43
3,10,caterpie,kanto,3,29,39,[bug],"[shield-dust, run-away]","[tackle, string-shot, snore, bug-bite, electro...","[eterna-forest-area, sinnoh-route-204-south-to...",...,Não encontrado.,False,False,False,45,30,35,20,20,45
4,13,weedle,kanto,3,32,39,"[bug, poison]","[shield-dust, run-away]","[poison-sting, string-shot, bug-bite, electroweb]","[eterna-forest-area, sinnoh-route-204-south-to...",...,Não encontrado.,False,False,False,40,35,30,20,20,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,1000,gholdengo,paldea,12,300,275,"[steel, ghost]","[good-as-gold, good-as-gold]","[thunder-punch, tackle, take-down, hyper-beam,...",[],...,gimmighoul,False,False,False,87,60,95,133,91,84
1021,1011,dipplin,paldea,4,97,170,"[grass, dragon]","[supersweet-syrup, gluttony, sticky-hold]","[body-slam, take-down, hyper-beam, growth, sol...",[],...,applin,False,False,False,80,80,110,95,80,40
1022,1013,sinistcha,paldea,2,22,178,"[grass, ghost]","[hospitality, heatproof]","[hyper-beam, absorb, mega-drain, solar-beam, s...",[],...,poltchageist,False,False,False,71,60,106,121,80,70
1023,1018,archaludon,paldea,20,600,300,"[steel, dragon]","[stamina, sturdy, stalwart]","[swords-dance, body-slam, take-down, double-ed...",[],...,duraludon,False,False,False,90,105,130,125,65,85


In [34]:
pokemon_data.columns

Index(['id', 'name', 'region', 'height', 'weight', 'base_experience', 'types',
       'abilities', 'moves', 'location_area_encounters', 'held_items',
       'base_happiness', 'color', 'capture_rate', 'evolves_from', 'is_baby',
       'is_mythical', 'is_legendary', 'hp', 'attack', 'defense',
       'special-attack', 'special-defense', 'speed'],
      dtype='object')

In [37]:
# Pokemons disponiveis
pokemon_data['id'].value_counts().sum()

1025

In [40]:
lendarios = pokemon_data.loc[pokemon_data['is_legendary'] == True]
display(lendarios)

,id,name,region,height,weight,base_experience,types,abilities,moves,location_area_encounters,...,evolves_from,is_baby,is_mythical,is_legendary,hp,attack,defense,special-attack,special-defense,speed
108,144,articuno,kanto,17,554,290,"[ice, flying]","[pressure, snow-cloak]","[razor-wind, gust, whirlwind, fly, headbutt, t...","[seafoam-islands-b4f, roaming-sinnoh-area, ult...",...,Não encontrado.,False,False,True,90,85,100,95,125,85
109,145,zapdos,kanto,16,526,290,"[electric, flying]","[pressure, static]","[razor-wind, whirlwind, fly, headbutt, take-do...","[kanto-route-10-area, power-plant-area, roamin...",...,Não encontrado.,False,False,True,90,90,85,125,90,100
110,146,moltres,kanto,20,600,290,"[fire, flying]","[pressure, flame-body]","[razor-wind, gust, wing-attack, whirlwind, fly...","[mt-silver-top, kanto-victory-road-2-2f, roami...",...,Não encontrado.,False,False,True,90,100,90,125,85,90
112,150,mewtwo,kanto,20,1220,340,[psychic],"[pressure, unnerve]","[mega-punch, pay-day, fire-punch, ice-punch, t...","[cerulean-cave-b1f, ultra-space-wilds-crag]",...,Não encontrado.,False,False,True,106,110,90,154,90,130
234,243,raikou,johto,19,1780,290,[electric],"[pressure, inner-focus]","[cut, headbutt, body-slam, take-down, double-e...","[roaming-kanto-area, roaming-johto-area, ultra...",...,Não encontrado.,False,False,True,90,85,75,115,100,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,1014,okidogi,paldea,18,922,278,"[poison, fighting]","[toxic-chain, guard-dog]","[fire-punch, ice-punch, thunder-punch, body-sl...",[],...,Não encontrado.,False,False,True,88,128,115,58,86,80
1007,1015,munkidori,paldea,10,122,278,"[poison, psychic]","[toxic-chain, frisk]","[scratch, psybeam, hyper-beam, toxic, confusio...",[],...,Não encontrado.,False,False,True,88,75,66,130,90,106
1008,1016,fezandipiti,paldea,14,301,278,"[poison, fairy]","[toxic-chain, technician]","[swords-dance, wing-attack, fly, double-kick, ...",[],...,Não encontrado.,False,False,True,88,91,82,70,125,99
1009,1017,ogerpon,paldea,12,398,275,[grass],[defiant],"[swords-dance, slam, vine-whip, double-kick, t...",[],...,Não encontrado.,False,False,True,80,120,84,60,96,110
